## Solving the MIS with Bloqade Python and Amazon Braket Hybrid Jobs

In this example we pass our parameterized pulse program into a classical optimizer in order to provide classical feedback for the next quantum program to run. The use case we will cover here is a pretty standard problem that maps very neatly onto the AHS architecture implemented with Neutral atoms. Because of the Rydberg blockade effect the ground state of a collection of atoms maps exactly to what is called the Maximum Independent Set (MIS) problem on geometric graphs. The MIS problem is a graph coloring problem where the goal is to find the largest set of nodes in a graph such that no two nodes are connected by an edge. This problem is NP-hard and has many applications in scheduling and resource allocation.

We refer the reader to [this](https://github.com/QuEraComputing/QuEra-braket-examples/blob/main/OptimizationTutorial/AWS_optimization_demo.ipynb) Notebook example for a more detailed explanation of the problem and how it maps onto AHS. For this blog post we will focus on the implementation of the hybrid algorithm using Bloqade and Braket Hybrid Jobs. 

Like most of the Bloqade programs we begin by importing the necessary components to build the program:

In [ ]:
import numpy as np
from bloqade import RB_C6, save, start, var
from bloqade.atom_arrangement import Square
from braket.devices import Devices
from braket.aws import AwsDevice

using the `AwsDevice` we can get some information about the capabilities of the device. For the sake of demonstration this example does not submit to quantum hardware immediately (although the option will be presented later). However, it is still useful to know the hardware capabilities in designing future programs. Note that Bloqade uses `rad/us` and `us` for energy and time units respectively while braket uses SI units, e.g. `rad/s` and `s`, hence the conversion of units below. 


In [ ]:
## Obtain hardware constraints for the Aquila Quantum Processing Unit (QPU)
## via `AwsDevice`
device = AwsDevice(Devices.QuEra.Aquila)
rydberg_properties = device.properties.paradigm.rydberg.rydbergGlobal

## Convert energy units to rad/us and time to us
detuning_max = float(rydberg_properties.detuningRange[1]) / 1e6
rabi_max = float(rydberg_properties.rabiFrequencyRange[1]) / 1e6
total_time = float(rydberg_properties.timeMax) * 1e6

For the particular problem we are studying we need to map the problem graph onto the atom arrangement. For more information we refer the reader to the notebook [example](https://github.com/QuEraComputing/QuEra-braket-examples/blob/main/OptimizationTutorial/AWS_optimization_demo.ipynb). 

In [ ]:
## make sure next nearest neighbors are blockaded
## unit disk minimum and maximum radii
Rmin = np.sqrt(2)  ## minimum unit disk radius
Rmax = 2  ## maximum unit disk radius
## choose geometric mean of Rmin and Rmax
Rudg = np.sqrt(Rmin * Rmax)

detuning_end = detuning_max / 4
blockade_radius = (RB_C6 / detuning_end) ** (1 / 6)
lattice_spacing = blockade_radius / Rudg

Now we can define the program. We will use the `var` function to define the parameters for the program. Also the cost function will involve calculating the final energy of tha atoms which can be obtained from the geometry of the program via the `rydberg_interaction` method.

In [ ]:
## define the time step for the detuning waveform,
## basically the time between allowable changes in detuning
dt = 0.4
n_steps = int(total_time / dt)
## create variables for the allowable changes in detuning.
## These variables allow us to parametrize our program and
detuning_vars = [var(f"d{i}") for i in range(n_steps - 1)]

# define the lattice size before defect insertion
L = 4
# set seed for geometry generation
rng = np.random.default_rng(1337)
program = (
    Square(L, lattice_spacing)
    .apply_defect_count(L**2 // 2, rng=rng)
    .remove_vacant_sites()
    .rydberg.detuning.uniform.piecewise_linear(
        n_steps * [dt], [-detuning_end] + detuning_vars + [detuning_end]
        # n_steps * [dt] creates [dt, dt, dt]
        # [-detuning_end] + detuning_vars + [detuning_end] creates [-detuning_end, d0, d1, ..., dN, detuning_end]
    )
    .amplitude.uniform.piecewise_linear(
        [0.1, total_time - 0.2, 0.1], [0, rabi_max, rabi_max, 0]
    )
)
# get atom register and interaction matrix
V_ij = program.parse_register().rydberg_interaction()

We need to build the infrastructure to do the hybrid job. There are many different tools available in `braket.jobs` that allow you to log the progress of your hybrid algorithm. Here we set up a simple class that wraps the cost function and log the progress of the algorithm. 

In [ ]:
from braket.jobs import (
    InstanceConfig,
    hybrid_job,
    save_job_checkpoint,
)
from braket.jobs.metrics import log_metric
from braket.jobs_data import PersistedJobDataFormat
from braket.tracking import Tracker


# define a wrapper for the cost function for reporting
class CostFuncWrapper:
    def __init__(self, backend, shots=10, **options):
        self.backend = backend
        self.options = options
        self.iterations = 0
        self.shots = shots
        self.prev_calls = {}
        self.task_tracker = Tracker().start()

    @staticmethod
    def cost_func(report):
        bitstrings = 1 - np.asarray(report.bitstrings(False))
        detuning_energies = -detuning_end * bitstrings.sum(axis=-1)
        interaction_energies = np.einsum(
            "ij, ...i, ...j-> ...", V_ij, bitstrings, bitstrings
        )

        total_energy = detuning_energies + interaction_energies
        # minimize the energy mean and standard deviation
        return total_energy.mean() + total_energy.std()

    def __call__(self, x):
        args = tuple(x)

        batch_task = self.backend.run(self.shots, args=args, **self.options)
        report = batch_task.report()

        save(batch_task, f"my-backend_results-{self.iterations}.json")

        self.prev_calls[args] = report
        return self.cost_func(report)

    def callback(self, state):
        args = tuple(state.x)
        self.iterations += 1
        bitstrings = 1 - np.asarray(self.prev_calls[args].bitstrings(False))
        detuning_energies = -detuning_end * bitstrings.sum(axis=-1)

        interaction_energies = np.einsum(
            "ij, ...i, ...j-> ...", V_ij, bitstrings, bitstrings
        )

        total_energy = detuning_energies + interaction_energies
        mean_energy = total_energy.mean()
        std_energy = total_energy.std()

        # Log metrics to display in Amazon Braket Console
        log_metric(
            iteration_number=self.iterations, value=state.fun, metric_name="loss"
        )
        log_metric(
            iteration_number=self.iterations,
            value=mean_energy,
            metric_name="mean energy",
        )
        log_metric(
            iteration_number=self.iterations,
            value=std_energy,
            metric_name="std energy",
        )

        # Also track the quantum task cost for Amazon Braket devices
        braket_task_cost = float(
            self.task_tracker.qpu_tasks_cost()
            + self.task_tracker.simulator_tasks_cost()
        )
        log_metric(
            metric_name="braket_cost",
            value=braket_task_cost,
            iteration_number=self.iterations,
        )

        # Save a checkpoint to resume the hybrid job from where you left off
        checkpoint_data = {"i": self.iterations, "args": args}
        # this function is from Braket
        save_job_checkpoint(
            checkpoint_data, data_format=PersistedJobDataFormat.PICKLED_V4
        )
        # end the job if the std energy is less than 5% of the mean energy
        # this indicates that the system is close to the ground state
        return abs(std_energy / mean_energy) < 0.05

While this is a lot to take in let us go through some important things. Firstly, you can generate a `Report` object from a Bloqade batch of tasks. This object provides some basic analysis methods common to neutral atom AHS. The one we make the most use of here is the `bitstrings` method which return a list of arrays that contains the shot results after executing the program. It takes a boolean argument that specifies whether or not to return the bitstrings of shots where there were atoms that did not successfully get put into position before the computation was executed. By default this filter is applied, but for this problem we want to include those shots in our analysis hence the `False` argument. 

Another thing to note is that our cost function not only contains the `mean` energy but also the standard deviation. The reason for this is because we are targeting an eigenstate of the final Hamiltonian which has no energy variance. This is a good way to check if the system is in the ground state. We use the ratio of the standard deviation to the mean energy as a stopping condition for the algorithm.

Finally, we have a callback function that is called after each iteration of the optimizer. This is where we can log the progress of the algorithm. We use the `Tracker` object to track the cost of the quantum tasks that are being executed. We also use the `log_metric` function to log the mean and standard deviation of the energy as well as the cost function of the quantum tasks.

Now we can define the function that will run the hybrid job. 

<div class="alert alert-block alert-warning"> 
The moment the function below (run_algo) is executed, it will be submitted to Amazon Braket.

For the sake of demonstration the quantum `device` it's targeting is set to `None` meaning it will not execute on quantum hardware and will instead just use the associated Amazon EC2 resource. This will still incur a cost, as will submitting to quantum hardware.

If you'd like to submit to our quantum machine instead of relying on Bloqade's emulator, just change the `device` and program backend as detailed in the comments.
</div>

In [ ]:
@hybrid_job(
        # set this to `Devices.QuEra.Aquila` if you'd like to run on the actual QPU
        device = None,
        dependencies="requirements.txt",  # install bloqade and scikit-optimize
        instance_config=InstanceConfig("ml.m5.large"),
)
def run_algo(assigned_program, n_calls=10, n_shots=10):
    from skopt import gp_minimize

    # Change `assigned_program.bloqade.python()` to `assigned_program.braket.aquila()`
    # along with the change in the device arn above to run on *Aquila*
    program_with_backend = assigned_program.bloqade.python()
    wrapped_cost_func = CostFuncWrapper(program_with_backend, shots=n_shots)

    n_params = len(program_with_backend.params.args_list)
    bounds = n_params * [(-detuning_max, detuning_max)]

    result = gp_minimize(
        wrapped_cost_func,
        bounds,
        callback=wrapped_cost_func.callback,
        n_calls=n_calls,
    )

    detuning_values = {var.name: val for var, val in zip(detuning_vars, result.x)}

    return detuning_values

We use the `hybrid_job` decorator to define the hybrid job. This decorator takes a `device_arn` argument which is the Amazon Resource Name (ARN) of the device you want to run the hybrid job on. There are some other options associated with the Amazon EC2 instance that is used to run the hybrid job. We will use the `InstanceConfig` object to specify the instance type and number of instances to use. `dependencies` points to a text file that contains the python dependencies needed to run this hybrid job. In this case we need `bloqade` and `scikit-optimize` in this text file. 

The function `run_algo` that is being decorated generates a backend object from the parametrized program we've created by the Bloqade API and must match the device ARN that is specified in the decorator. `n_calls` is the total number of iterations of the optimizer and `shots` is the number of shots to use for each iteration. `options` is a dictionary of options that are passed to the `run` method of the backend. The return value of the function is the final value of the parameters that were optimized. 

To run the hybrid job we simply call the `run_algo` function with the assigned program and the device ARN.

In [ ]:
optimal_params = run_algo(program.args(detuning_vars), n_calls=10, n_shots=10)
assigned_program = program.assign(**optimal_params.result())

We will have to wait for the Hybrid Job to finish running but afterwards you can assign fixed values to the program now that we know what they optimally are and remove the need to apply them on the fly as we have before. You can visualize the final waveform with the built-in Bokeh visualiztions Bloqade has with `assigned_program.show()` which should open a window in your browser.